In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd

Mounted at /content/drive


In [11]:
os.chdir("./tropical-cyclones-xai-shared")
!ls

build_datasets	datasets  models


## Target Dataframe

In [14]:
ls

build_datasets/  datasets/  models/


In [12]:
def attribute_stepped(att, max_steps):
  att_list = []
  for i in range(max_steps):
    att_list.append('{}-{}'.format(att,i))
  return att_list

In [15]:
from datetime import datetime

df_target = pd.read_csv('./datasets/target_labels/tc_act_sind.csv')

df_target = df_target.rename(columns={'Unnamed: 0': 'DATE'})
df_target = df_target[11:]
df_target = df_target.reset_index(drop=True)
df_target

,DATE,S.IndAll,S.IndGen
0,1980-01-12,0.00,0.0
1,1980-01-13,0.00,0.0
2,1980-01-14,0.00,0.0
3,1980-01-15,0.00,0.0
4,1980-01-16,0.00,0.0
...,...,...,...
15411,2022-03-23,0.50,1.0
15412,2022-03-24,1.00,0.0
15413,2022-03-25,1.00,0.0
15414,2022-03-26,1.00,0.0


In [16]:
def rename_labels(df_to_rename, prefix):
  old_labels = df_to_rename.columns
  new_labels = []
  for s in old_labels:
      new_labels.append(prefix + s)
  df_to_rename.columns = new_labels
  return df_to_rename

In [17]:
df_pressure = pd.read_csv('./build_datasets/csv/16_zones/pressure_10D.csv')
df_pressure = df_pressure.drop(['DATE', 'S.IndAll'], axis=1)
df_pressure = rename_labels(df_pressure, 'P_')
df_pressure

,P_Mean_Z1-0,P_Mean_Z1-1,P_Mean_Z1-2,P_Mean_Z1-3,P_Mean_Z1-4,P_Mean_Z1-5,P_Mean_Z1-6,P_Mean_Z1-7,P_Mean_Z1-8,P_Mean_Z1-9,...,P_Std_Z16-1,P_Std_Z16-2,P_Std_Z16-3,P_Std_Z16-4,P_Std_Z16-5,P_Std_Z16-6,P_Std_Z16-7,P_Std_Z16-8,P_Std_Z16-9,P_Std_Z16-10
0,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,101174.726566,101211.364303,101128.555878,101129.258287,...,171.141902,151.797225,101.009716,145.169986,141.886689,138.845057,167.187538,175.589188,179.948952,210.016876
1,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,101174.726566,101211.364303,101128.555878,...,220.919479,171.141902,151.797225,101.009716,145.169986,141.886689,138.845057,167.187538,175.589188,179.948952
2,101119.437077,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,101174.726566,101211.364303,...,219.035164,220.919479,171.141902,151.797225,101.009716,145.169986,141.886689,138.845057,167.187538,175.589188
3,101184.010287,101119.437077,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,101174.726566,...,197.181702,219.035164,220.919479,171.141902,151.797225,101.009716,145.169986,141.886689,138.845057,167.187538
4,101230.576023,101184.010287,101119.437077,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,...,197.554465,197.181702,219.035164,220.919479,171.141902,151.797225,101.009716,145.169986,141.886689,138.845057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,101172.332563,100880.621576,100914.824882,101072.576001,101049.467974,...,271.491184,280.268464,275.884089,277.667707,274.820593,223.716394,205.052190,252.764845,211.859014,100.935979
15412,101239.688484,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,101172.332563,100880.621576,100914.824882,101072.576001,...,235.514894,271.491184,280.268464,275.884089,277.667707,274.820593,223.716394,205.052190,252.764845,211.859014
15413,101133.947820,101239.688484,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,101172.332563,100880.621576,100914.824882,...,191.498356,235.514894,271.491184,280.268464,275.884089,277.667707,274.820593,223.716394,205.052190,252.764845
15414,101032.107744,101133.947820,101239.688484,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,101172.332563,100880.621576,...,193.623679,191.498356,235.514894,271.491184,280.268464,275.884089,277.667707,274.820593,223.716394,205.052190


In [19]:
df_vor_850hPa = pd.read_csv('./build_datasets/csv/16_zones/vorticity_850hPa_10D.csv')
df_vor_550hPa = pd.read_csv('./build_datasets/csv/16_zones/vorticity_550hPa_10D.csv')
df_vor_250hPa = pd.read_csv('./build_datasets/csv/16_zones/vorticity_250hPa_10D.csv')

df_vor_850hPa = df_vor_850hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_vor_550hPa = df_vor_550hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_vor_250hPa = df_vor_250hPa.drop(['DATE', 'S.IndAll'], axis=1)

df_vor_850hPa = rename_labels(df_vor_850hPa, 'Vor_850hPa_')
df_vor_550hPa = rename_labels(df_vor_550hPa, 'Vor_550hPa_')
df_vor_250hPa = rename_labels(df_vor_250hPa, 'Vor_250hPa_')

In [20]:
df_wind_1000hPa = pd.read_csv('./build_datasets/csv/16_zones/wind_speed_1000hPa_10D.csv')
df_wind_850hPa  = pd.read_csv('./build_datasets/csv/16_zones/wind_speed_850hPa_10D.csv')
df_wind_300hPa  = pd.read_csv('./build_datasets/csv/16_zones/wind_speed_300hPa_10D.csv')
df_wind_10m     = pd.read_csv('./build_datasets/csv/16_zones/wind_speed_10m_10D.csv')
df_wind_gust    = pd.read_csv('./build_datasets/csv/16_zones/wind_gust_10D.csv')

df_wind_1000hPa = df_wind_1000hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_wind_850hPa  = df_wind_850hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_wind_300hPa  = df_wind_300hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_wind_10m     = df_wind_10m.drop(['DATE', 'S.IndAll'], axis=1)
df_wind_gust    = df_wind_gust.drop(['DATE', 'S.IndAll'], axis=1)

df_wind_1000hPa = rename_labels(df_wind_1000hPa, 'Wind_1000hPa_')
df_wind_850hPa  = rename_labels(df_wind_850hPa, 'Wind_850hPa_')
df_wind_300hPa  = rename_labels(df_wind_300hPa, 'Wind_300hPa_')
df_wind_10m     = rename_labels(df_wind_10m, 'Wind_10m_')
df_wind_gust    = rename_labels(df_wind_gust, 'Wind_Gust_')

In [21]:
df_precipitation = pd.read_csv('./build_datasets/csv/16_zones/precipitation_10D.csv')
df_precipitation = df_precipitation.drop(['DATE', 'S.IndAll'], axis=1)
df_precipitation = rename_labels(df_precipitation, 'Prec_')

df_precipitation

,Prec_Mean_Z1-0,Prec_Mean_Z1-1,Prec_Mean_Z1-2,Prec_Mean_Z1-3,Prec_Mean_Z1-4,Prec_Mean_Z1-5,Prec_Mean_Z1-6,Prec_Mean_Z1-7,Prec_Mean_Z1-8,Prec_Mean_Z1-9,...,Prec_Std_Z16-1,Prec_Std_Z16-2,Prec_Std_Z16-3,Prec_Std_Z16-4,Prec_Std_Z16-5,Prec_Std_Z16-6,Prec_Std_Z16-7,Prec_Std_Z16-8,Prec_Std_Z16-9,Prec_Std_Z16-10
0,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,0.000049,0.000139,0.000059,0.000120,...,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044,0.000057,0.000066,0.000088
1,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,0.000049,0.000139,0.000059,...,0.000080,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044,0.000057,0.000066
2,0.000026,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,0.000049,0.000139,...,0.000062,0.000080,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044,0.000057
3,0.000051,0.000026,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,0.000049,...,0.000052,0.000062,0.000080,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044
4,0.000085,0.000051,0.000026,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,...,0.000056,0.000052,0.000062,0.000080,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,0.000252,0.000016,0.000014,0.000011,...,0.000079,0.000078,0.000040,0.000044,0.000070,0.000049,0.000047,0.000090,0.000064,0.000091
15412,0.000049,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,0.000252,0.000016,0.000014,...,0.000065,0.000079,0.000078,0.000040,0.000044,0.000070,0.000049,0.000047,0.000090,0.000064
15413,0.000022,0.000049,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,0.000252,0.000016,...,0.000057,0.000065,0.000079,0.000078,0.000040,0.000044,0.000070,0.000049,0.000047,0.000090
15414,0.000028,0.000022,0.000049,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,0.000252,...,0.000061,0.000057,0.000065,0.000079,0.000078,0.000040,0.000044,0.000070,0.000049,0.000047


In [22]:
df_temperature_200hPa = pd.read_csv('./build_datasets/csv/16_zones/temperature_200hPa_10D.csv')
df_temperature_300hPa = pd.read_csv('./build_datasets/csv/16_zones/temperature_300hPa_10D.csv')
df_temperature_550hPa = pd.read_csv('./build_datasets/csv/16_zones/temperature_550hPa_10D.csv')
df_temperature_1000hPa = pd.read_csv('./build_datasets/csv/16_zones/temperature_1000hPa_10D.csv')

df_temperature_200hPa = df_temperature_200hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_temperature_300hPa = df_temperature_300hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_temperature_550hPa = df_temperature_550hPa.drop(['DATE', 'S.IndAll'], axis=1)
df_temperature_1000hPa = df_temperature_1000hPa.drop(['DATE', 'S.IndAll'], axis=1)

df_temperature_200hPa = rename_labels(df_temperature_200hPa, 'T_200hPa_')
df_temperature_300hPa = rename_labels(df_temperature_300hPa, 'T_300hPa_')
df_temperature_550hPa = rename_labels(df_temperature_550hPa, 'T_550hPa_')
df_temperature_1000hPa = rename_labels(df_temperature_1000hPa, 'T_1000hPa_')

df_temperature_200hPa

,T_200hPa_Mean_Z1-0,T_200hPa_Mean_Z1-1,T_200hPa_Mean_Z1-2,T_200hPa_Mean_Z1-3,T_200hPa_Mean_Z1-4,T_200hPa_Mean_Z1-5,T_200hPa_Mean_Z1-6,T_200hPa_Mean_Z1-7,T_200hPa_Mean_Z1-8,T_200hPa_Mean_Z1-9,...,T_200hPa_Std_Z16-1,T_200hPa_Std_Z16-2,T_200hPa_Std_Z16-3,T_200hPa_Std_Z16-4,T_200hPa_Std_Z16-5,T_200hPa_Std_Z16-6,T_200hPa_Std_Z16-7,T_200hPa_Std_Z16-8,T_200hPa_Std_Z16-9,T_200hPa_Std_Z16-10
0,220.031599,220.107611,219.506949,219.082111,219.189421,219.372680,219.227063,218.873189,218.950656,219.348074,...,0.686731,0.884226,0.994397,1.095202,1.183172,1.368140,1.528161,1.090290,0.719526,0.686094
1,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,219.372680,219.227063,218.873189,218.950656,...,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.368140,1.528161,1.090290,0.719526
2,218.708478,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,219.372680,219.227063,218.873189,...,1.357856,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.368140,1.528161,1.090290
3,218.715620,218.708478,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,219.372680,219.227063,...,1.713481,1.357856,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.368140,1.528161
4,219.622547,218.715620,218.708478,219.336468,220.031599,220.107611,219.506949,219.082111,219.189421,219.372680,...,2.471615,1.713481,1.357856,1.874089,0.686731,0.884226,0.994397,1.095202,1.183172,1.368140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,220.377924,220.075790,219.525139,219.677996,...,1.093564,1.462673,1.735088,1.440886,1.241267,1.170320,1.668688,0.901483,0.557791,0.794303
15412,218.772251,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,220.377924,220.075790,219.525139,...,1.438795,1.093564,1.462673,1.735088,1.440886,1.241267,1.170320,1.668688,0.901483,0.557791
15413,218.775710,218.772251,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,220.377924,220.075790,...,0.971513,1.438795,1.093564,1.462673,1.735088,1.440886,1.241267,1.170320,1.668688,0.901483
15414,219.746559,218.775710,218.772251,219.319951,219.167548,219.232487,219.910158,219.672019,219.828496,220.377924,...,0.722973,0.971513,1.438795,1.093564,1.462673,1.735088,1.440886,1.241267,1.170320,1.668688


In [23]:
df_air_density = pd.read_csv('./build_datasets/csv/16_zones/air_density_10D.csv')
df_air_density = df_air_density.drop(['DATE', 'S.IndAll'], axis=1)
df_air_density = rename_labels(df_air_density, 'Air_Density_')

df_air_density

,Air_Density_Mean_Z1-0,Air_Density_Mean_Z1-1,Air_Density_Mean_Z1-2,Air_Density_Mean_Z1-3,Air_Density_Mean_Z1-4,Air_Density_Mean_Z1-5,Air_Density_Mean_Z1-6,Air_Density_Mean_Z1-7,Air_Density_Mean_Z1-8,Air_Density_Mean_Z1-9,...,Air_Density_Std_Z16-1,Air_Density_Std_Z16-2,Air_Density_Std_Z16-3,Air_Density_Std_Z16-4,Air_Density_Std_Z16-5,Air_Density_Std_Z16-6,Air_Density_Std_Z16-7,Air_Density_Std_Z16-8,Air_Density_Std_Z16-9,Air_Density_Std_Z16-10
0,1.138818,1.138890,1.140092,1.142138,1.142000,1.140919,1.140491,1.140551,1.141045,1.140072,...,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481,0.006592,0.007094,0.005822
1,1.138880,1.138818,1.138890,1.140092,1.142138,1.142000,1.140919,1.140491,1.140551,1.141045,...,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481,0.006592,0.007094
2,1.140459,1.138880,1.138818,1.138890,1.140092,1.142138,1.142000,1.140919,1.140491,1.140551,...,0.008812,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481,0.006592
3,1.141769,1.140459,1.138880,1.138818,1.138890,1.140092,1.142138,1.142000,1.140919,1.140491,...,0.008131,0.008812,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481
4,1.142220,1.141769,1.140459,1.138880,1.138818,1.138890,1.140092,1.142138,1.142000,1.140919,...,0.008338,0.008131,0.008812,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,1.139390,1.138829,1.136585,1.137211,1.139110,1.138227,1.134916,1.135812,1.139503,1.138537,...,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228,0.010348,0.009987,0.007693
15412,1.136788,1.139390,1.138829,1.136585,1.137211,1.139110,1.138227,1.134916,1.135812,1.139503,...,0.008156,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228,0.010348,0.009987
15413,1.134194,1.136788,1.139390,1.138829,1.136585,1.137211,1.139110,1.138227,1.134916,1.135812,...,0.007617,0.008156,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228,0.010348
15414,1.134000,1.134194,1.136788,1.139390,1.138829,1.136585,1.137211,1.139110,1.138227,1.134916,...,0.007894,0.007617,0.008156,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228


In [24]:
df_tot = pd.merge(df_target, df_pressure, left_index=True, right_index=True)

df_tot = pd.merge(df_tot, df_vor_850hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_vor_550hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_vor_250hPa, left_index=True, right_index=True)

df_tot = pd.merge(df_tot, df_wind_gust, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_wind_10m, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_wind_1000hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_wind_850hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_wind_300hPa, left_index=True, right_index=True)

df_tot = pd.merge(df_tot, df_temperature_1000hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_temperature_550hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_temperature_300hPa, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_temperature_200hPa, left_index=True, right_index=True)

df_tot = pd.merge(df_tot, df_precipitation, left_index=True, right_index=True)
df_tot = pd.merge(df_tot, df_air_density, left_index=True, right_index=True)

columns = df_tot.columns.values
print(columns[2::88])

df_tot

['S.IndGen' 'P_Std_Z4-10' 'P_Std_Z8-10' 'P_Std_Z12-10' 'P_Std_Z16-10'
 'Vor_850hPa_Std_Z4-10' 'Vor_850hPa_Std_Z8-10' 'Vor_850hPa_Std_Z12-10'
 'Vor_850hPa_Std_Z16-10' 'Vor_550hPa_Std_Z4-10' 'Vor_550hPa_Std_Z8-10'
 'Vor_550hPa_Std_Z12-10' 'Vor_550hPa_Std_Z16-10' 'Vor_250hPa_Std_Z4-10'
 'Vor_250hPa_Std_Z8-10' 'Vor_250hPa_Std_Z12-10' 'Vor_250hPa_Std_Z16-10'
 'Wind_Gust_Std_Z4-10' 'Wind_Gust_Std_Z8-10' 'Wind_Gust_Std_Z12-10'
 'Wind_Gust_Std_Z16-10' 'Wind_10m_Std_Z4-10' 'Wind_10m_Std_Z8-10'
 'Wind_10m_Std_Z12-10' 'Wind_10m_Std_Z16-10' 'Wind_1000hPa_Std_Z4-10'
 'Wind_1000hPa_Std_Z8-10' 'Wind_1000hPa_Std_Z12-10'
 'Wind_1000hPa_Std_Z16-10' 'Wind_850hPa_Std_Z4-10' 'Wind_850hPa_Std_Z8-10'
 'Wind_850hPa_Std_Z12-10' 'Wind_850hPa_Std_Z16-10' 'Wind_300hPa_Std_Z4-10'
 'Wind_300hPa_Std_Z8-10' 'Wind_300hPa_Std_Z12-10' 'Wind_300hPa_Std_Z16-10'
 'T_1000hPa_Std_Z4-10' 'T_1000hPa_Std_Z8-10' 'T_1000hPa_Std_Z12-10'
 'T_1000hPa_Std_Z16-10' 'T_550hPa_Std_Z4-10' 'T_550hPa_Std_Z8-10'
 'T_550hPa_Std_Z12-10' 'T_550

,DATE,S.IndAll,S.IndGen,P_Mean_Z1-0,P_Mean_Z1-1,P_Mean_Z1-2,P_Mean_Z1-3,P_Mean_Z1-4,P_Mean_Z1-5,P_Mean_Z1-6,...,Air_Density_Std_Z16-1,Air_Density_Std_Z16-2,Air_Density_Std_Z16-3,Air_Density_Std_Z16-4,Air_Density_Std_Z16-5,Air_Density_Std_Z16-6,Air_Density_Std_Z16-7,Air_Density_Std_Z16-8,Air_Density_Std_Z16-9,Air_Density_Std_Z16-10
0,1980-01-12,0.00,0.0,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,101174.726566,...,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481,0.006592,0.007094,0.005822
1,1980-01-13,0.00,0.0,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,101080.710310,...,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481,0.006592,0.007094
2,1980-01-14,0.00,0.0,101119.437077,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,101147.053416,...,0.008812,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481,0.006592
3,1980-01-15,0.00,0.0,101184.010287,101119.437077,101027.665055,100998.387751,101075.154767,101097.224290,101223.690588,...,0.008131,0.008812,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873,0.005481
4,1980-01-16,0.00,0.0,101230.576023,101184.010287,101119.437077,101027.665055,100998.387751,101075.154767,101097.224290,...,0.008338,0.008131,0.008812,0.009503,0.006282,0.006422,0.004653,0.004402,0.003585,0.004873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,2022-03-23,0.50,1.0,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,101172.332563,100880.621576,...,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228,0.010348,0.009987,0.007693
15412,2022-03-24,1.00,0.0,101239.688484,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,101172.332563,...,0.008156,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228,0.010348,0.009987
15413,2022-03-25,1.00,0.0,101133.947820,101239.688484,101261.177785,101059.680774,101048.126390,101124.522317,101241.370775,...,0.007617,0.008156,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228,0.010348
15414,2022-03-26,1.00,0.0,101032.107744,101133.947820,101239.688484,101261.177785,101059.680774,101048.126390,101124.522317,...,0.007894,0.007617,0.008156,0.009370,0.009009,0.009887,0.010371,0.010228,0.008420,0.009228


In [ ]:
df_tot.to_csv('./new_datasets/ERA5_16zones_avg_std_10D_NoWindDirection.csv', index=False)